### Dependencies

In [ ]:
pip install py-AutoClean datasets cleanlab scikit-learn jenga ftfy pandas numpy setuptools nltk ucimlrepo category_encoders -q

In [ ]:
import sys
print(sys.executable)
from AutoClean import AutoClean

In [ ]:
import sklearn
sklearn.set_config(enable_metadata_routing=True)
import sys
import os

sys.path.append(os.path.abspath(".."))


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.text import BrokenCharacters
import os
import ftfy
from category_encoders import TargetEncoder
from datasets import load_dataset
from sklearn.model_selection import train_test_split

### Load the data - Amazon

In [ ]:
from pathlib import Path
print("Loading Amazon Reviews 2023 (All Beauty)...")

BASE_DIR =  Path.cwd().parent / "datasets"
AMAZON_DIR = os.path.join(BASE_DIR, "amazon_reviews_2023_all_beauty")
os.makedirs(AMAZON_DIR, exist_ok=True)

try:
    dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023",
        "raw_review_All_Beauty",
        split="full",
        streaming=False
    )
    
    df = dataset.to_pandas()
    
except Exception as e:
    print(f"Error loading from HF: {e}")


print(f"✅ Loaded {len(df)} reviews")
print(f"Columns: {df.columns.tolist()}")
print(df.head())

# Sample 50k for faster local processing
df_sample = df
sample_path = os.path.join(AMAZON_DIR, "sample_50k.csv")
df_sample.to_csv(sample_path, index=False)
print(f"✅ Saved sample to: {sample_path}")

In [ ]:
CORRUPT_DIR = os.path.join(AMAZON_DIR, "corrupted_batches")
ARTIFACTS_DIR = os.path.join(BASE_DIR, "artifacts")

os.makedirs(AMAZON_DIR, exist_ok=True)
os.makedirs(CORRUPT_DIR, exist_ok=True)
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

### Feature extraction and model - Amazon

In [ ]:
dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_All_Beauty",
    split="full",
    streaming=False
)
df = dataset.to_pandas()
print(f"✅ Loaded {len(df)} reviews")

if "text" not in df.columns:
    df["text"] = (df.get("title", "").fillna("").astype(str) + " " + 
                  df.get("body", "").fillna("").astype(str)).str.strip()
    
from sklearn.utils import resample

# Find the smallest class count
min_count = df['rating'].value_counts().min()

# Downsample each class to the minimum count
df = (
    df.groupby('rating', group_keys=False)
      .apply(lambda x: resample(x, replace=False, n_samples=20000, random_state=42))
)

df = df.dropna(subset=["rating", "text"])
df = df[df["text"].str.len() > 0]

df["label"] = df["rating"].astype(int)

print(f"\n📊 Data Summary:")
print(f"   Total samples: {len(df)}")
print(f"   Label distribution: {df['label'].value_counts().to_dict()}")

X = df["text"].values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  
    random_state=42,
    stratify=y
)

print(f"\n✅ Train/Test Split:")
print(f"   Train: {len(X_train)} samples")
print(f"   Test: {len(X_test)} samples")

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def preprocess_review(text):
    """
    Preprocess Amazon review text using NLTK
    
    Transforms raw review → clean tokens ready for TF-IDF
    """
    # Handle missing/null values
    if pd.isna(text) or text == "nan":
        return ""
    
    text = str(text).lower()
    
    # Step 1: Remove URLs (users sometimes paste links)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Step 2: Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Step 3: Remove HTML tags (if any)
    text = re.sub(r'<[^>]+>', '', text)
    
    # Step 4: Remove special characters but keep basic punctuation
    # (punctuation can carry sentiment: "!" and "?" matter for reviews)
    text = re.sub(r'[^a-zA-Z0-9\s!?.,\'-]', '', text)
    
    # Step 5: Tokenization - split into words
    tokens = word_tokenize(text)
    
    # Step 6: Remove stopwords (common words: "the", "a", "is", "and")
    # These add noise and don't help predict ratings
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Step 7: Lemmatization - reduce words to root form
    # Examples: "running" → "run", "better" → "good", "worst" → "bad"
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Step 8: Remove single characters (leftover noise like "i", "a")
    tokens = [token for token in tokens if len(token) > 1]
    
    return ' '.join(tokens)


In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
text_features = FeatureUnion([
    ("word_tfidf", TfidfVectorizer(
        preprocessor=preprocess_review,
        stop_words='english',
        ngram_range=(1, 2),
        min_df=2,
        max_features=10000,
        sublinear_tf=True,
    )),
    ("char_tfidf", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        max_features=3000,
        sublinear_tf=True,
    ))
])

In [ ]:
def build_model():
    """Create TF-IDF + LogisticRegression pipeline"""
    return Pipeline([
            ("features", text_features),
            ("clf", LogisticRegression(
                max_iter=2000,
                C=1.0,
                random_state=42,
            ))
        ])

In [ ]:
from sklearn.metrics import mean_absolute_error

baseline_model = build_model()
baseline_model.fit(X_train, y_train)

# Evaluate baseline
y_pred_baseline = baseline_model.predict(X_test)
baseline_acc = accuracy_score(y_test, y_pred_baseline)

y_pred = np.clip(np.round(y_pred_baseline), 1, 5)

mae = mean_absolute_error(y_test, y_pred)

print(f"\n✅ Baseline Model Trained")
print(f"📊 MAE: {mae:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))

# Save baseline accuracy
baseline_metrics = {
    "accuracy": baseline_acc,
    "predictions": y_pred_baseline
}

### Load the Data - Clothing Fit

In [ ]:
df_clothing = pd.read_json("../data/raw/renttherunway_final_data.json", lines=True)
print(df_clothing.head)

### Feature Extraction and Model - Clothing Fit

In [ ]:
df_fit = df_clothing.dropna(subset=['fit'])
if "text" not in df_fit.columns:
    df_fit["text"] = (
        df_fit.get("review_text", "").fillna("").astype(str)
    ).str.strip()

n_per_fit = 20000

sample_fit = (
    df_fit
    .groupby('fit', group_keys=False)
    .apply(lambda x: x.sample(
        n=min(len(x), n_per_fit),
        random_state=42
    ))
)
sample_fit["label"] = sample_fit["fit"]


df_rating = df_clothing.dropna(subset=['rating'])

sample_rating = (
    df_fit
    .groupby('rating', group_keys=False)
    .apply(lambda x: x.sample(
        n=min(len(x), n_per_fit),
        random_state=42
    ))
)

In [ ]:
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    sample_fit['review_text'], sample_fit['fit'], test_size=0.2, random_state=42
)

In [ ]:
baseline_model_f = build_model()
baseline_model_f.fit(X_train_f, y_train_f)

In [ ]:
y_baseline_f = baseline_model_f.predict(X_test_f)
baseline_acc = accuracy_score(y_test_f, y_baseline_f)
print(baseline_acc)

In [ ]:
df_clothing = df_clothing.dropna(subset=['review_text', 'rating'])

df_clothing = df_clothing.reset_index(drop=True)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    sample_rating['review_text'], sample_rating['rating'], test_size=0.2, random_state=42
)

In [ ]:
baseline_model_f = build_model()
baseline_model_f.fit(X_train_f, y_train_f)

In [ ]:
y_baseline_f = baseline_model_f.predict(X_test_f)

mae = mean_absolute_error(y_test_f, y_baseline_f)

print(f"\n✅ Baseline Model Trained")
print(f"📊 MAE: {mae:.4f}")

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "clf__C": [1.0, 0.1], 
}

gridsearch = GridSearchCV(
    baseline_model_f,
    param_grid=param_grid,
    cv=5
) 
gridsearch.fit(X_train_f, y_train_f)
y_pred = gridsearch.predict(X_test_f)

print(mean_absolute_error(y_test_f, y_pred))

### Adult Income Classification

In [ ]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

df = pd.read_csv("adult_for_manual_edit.csv")

target_column = df.columns[-1]  
X = df.drop(columns=[target_column]).copy()
y = df[[target_column]].copy()

numeric_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
X[numeric_features] = X[numeric_features].apply(pd.to_numeric, errors='coerce')

categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    sparse_threshold=0
)

clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            random_state=42
        ))
    ])

y_test = y.values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X.fillna('0'), y, test_size=0.3, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy on corrupted CSV: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


In [ ]:
print(X)

In [ ]:
import random
import csv

# Input / output CSV paths
input_csv = "adult_for_manual_edit.csv"
output_csv = "adult_corrupted_text.csv"

# Parameters
fraction_corrupt = 0.5   # 10% of numeric cells will be corrupted
chars_to_inject = ['#','@','!','x','a']
negate_fraction = 0.1   # fraction of numeric cells to make negative

# Read the CSV lines
with open(input_csv, newline='', encoding='utf-8') as f:
    reader = list(csv.reader(f))
    header = reader[0]
    rows = reader[1:]

# Identify numeric columns (basic heuristic: first row parseable as float)
numeric_cols = []
for i, col in enumerate(header):
    try:
        float(rows[0][i])
        numeric_cols.append(i)
    except:
        continue

# Corrupt rows
for row in rows:
    for col_idx in numeric_cols:
        cell = row[col_idx]
        try:
            val = float(cell)
        except:
            continue  # skip non-numeric already

        # --- 1) Negate some values ---
        if random.random() < negate_fraction:
            val = -abs(val)

        # --- 2) Scale by random factor ---
        if random.random() < fraction_corrupt:
            factor = random.uniform(1.5, 3.0)
            val = val * factor

        # --- 3) Replace some values with '?' ---
        if random.random() < fraction_corrupt:
            row[col_idx] = '?'
            continue  # skip other edits

        # --- 4) Inject random character into number ---
        if random.random() < fraction_corrupt:
            row[col_idx] = str(val) + random.choice(chars_to_inject)
        else:
            row[col_idx] = str(val)

# Write out corrupted CSV
with open(output_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print(f"✅ Corrupted CSV saved to {output_csv}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# --- Read corrupted CSV ---
import pandas as pd
from sklearn.utils import resample

# --- 1) Load corrupted CSV ---
df = pd.read_csv("adult_corrupted_text.csv")

# --- 2) Split features / target ---
target_column = df.columns[-1]  # assume last column is the target
X = df.drop(columns=[target_column])
y = df[target_column]

# --- 3) Combine X and y for resampling ---
train = X.copy()
train['target'] = y

# --- 4) Separate classes ---
class_1 = train[train['target'] == '<=50K']
class_2 = train[train['target'] == '>50K']

# --- 5) Resample to 20k rows each ---
class_1_resampled = resample(class_1,
                             replace=False,
                             n_samples=10000,
                             random_state=42)

class_2_resampled = resample(class_2,
                             replace=False,
                             n_samples=10000,
                             random_state=42)

# --- 6) Combine resampled data ---
df_balanced = pd.concat([class_1_resampled, class_2_resampled]).sample(frac=1, random_state=42)  # shuffle

# --- 7) Split back into X and y ---
X = df_balanced.drop(columns=['target'])
y = df_balanced['target']
# Minimal numeric coercion for corrupted numbers
numeric_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
for col in numeric_features:
    X[col] = pd.to_numeric(X[col], errors='coerce')

categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']

# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], sparse_threshold=0)

clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_test = X_test.fillna('0')
X_train = X_train.fillna('0')
# y_test = y_test.loc[X_test.index]

clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)

print(f"Accuracy on corrupted CSV: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

def fit_adult(X_a, y_a):

    clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', HistGradientBoostingClassifier(
            learning_rate=0.06,
            max_depth=5,
            random_state=42
        ))
    ])

    y_test = y.values.ravel()

    X_train, X_test, y_train, y_test = train_test_split(X_a, y_a, test_size=0.3, random_state=42)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    return y_test, y_pred

y = y.loc[X.index]
y_test, y_pred = fit_adult(X, y)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### Data Corruption

In [ ]:
"""
Define individual corruption methods
"""

def apply_missing_values(df, fraction=0.30):
    """Batch 01: Missing Values in text"""
    df = df.copy()
    mv = MissingValues(column="text", fraction=fraction, missingness="MCAR")
    return mv.transform(df)

def apply_broken_characters(df, fraction=0.25):
    """Batch 02: Broken Characters in text"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=fraction)
    return bc.transform(df)

def apply_swapped_text(df, fraction=0.20):
    """Batch 03: Swapped text values"""
    df = df.copy()
    sv = SwappedValues(column="text", fraction=fraction)
    return sv.transform(df)

def apply_missing_labels(df, fraction=0.15):
    """Batch 04: Missing Labels"""
    df = df.copy()
    mv = MissingValues(column="label", fraction=fraction, missingness="MCAR")
    return mv.transform(df)

def apply_swapped_labels(df, fraction=0.12):
    """Batch 05: Swapped Labels"""
    df = df.copy()
    sv = SwappedValues(column="label", fraction=fraction)
    return sv.transform(df)

def apply_combined_text_corruption(df):
    """Batch 06: Broken Chars (10%) + Missing (8%)"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=0.10)
    df = bc.transform(df)
    mv = MissingValues(column="text", fraction=0.08, missingness="MCAR")
    return mv.transform(df)

def apply_combined_text_labels(df):
    """Batch 07: Swapped Text (15%) + Swapped Labels (8%)"""
    df = df.copy()
    sv_text = SwappedValues(column="text", fraction=0.15)
    df = sv_text.transform(df)
    sv_label = SwappedValues(column="label", fraction=0.08)
    return sv_label.transform(df)

def apply_heavy_missing(df):
    """Batch 08: Heavy Missing - Text (25%) + Labels (10%)"""
    df = df.copy()
    mv_text = MissingValues(column="text", fraction=0.25, missingness="MCAR")
    df = mv_text.transform(df)
    mv_label = MissingValues(column="label", fraction=0.10, missingness="MCAR")
    return mv_label.transform(df)

def apply_all_corruptions(df):
    """Batch 09: All - Broken (8%) + Swapped (10%) + Missing (5%)"""
    df = df.copy()
    bc = BrokenCharacters(column="text", fraction=0.08)
    df = bc.transform(df)
    sv_text = SwappedValues(column="text", fraction=0.10)
    df = sv_text.transform(df)
    mv_text = MissingValues(column="text", fraction=0.05, missingness="MCAR")
    df = mv_text.transform(df)
    sv_label = SwappedValues(column="label", fraction=0.05)
    return sv_label.transform(df)

print("✅ Corruption functions defined")

In [ ]:
df_corrupt_source = df.iloc[len(X_train) + len(X_test):][["text", "label"]].copy()
if len(df_corrupt_source) < 5000:
    df_corrupt_source = df.sample(n=5000, random_state=42).reset_index(drop=True)
else:
    df_corrupt_source = df_corrupt_source.sample(n=5000, random_state=42).reset_index(drop=True)

batches_config = [
    ("01_missing", apply_missing_values, {}),
    ("02_broken_chars", apply_broken_characters, {}),
    ("03_swapped", apply_swapped_text, {}),
    ("04_missing_labels", apply_missing_labels, {}),
    ("05_swapped_labels", apply_swapped_labels, {}),
    ("06_combined_text", apply_combined_text_corruption, {}),
    ("07_combined_both", apply_combined_text_labels, {}),
    ("08_heavy_missing", apply_heavy_missing, {}),
    ("09_all_corruptions", apply_all_corruptions, {}),
]

corrupted_batches = {}

for batch_name, corruption_fn, kwargs in batches_config:
    print(f"\n🔧 Batch {batch_name.split('_')[0]}: {batch_name.replace('_', ' ').title()}")
    df_batch = corruption_fn(df_corrupt_source, **kwargs)
    batch_path = os.path.join(CORRUPT_DIR, f"batch_{batch_name}.csv")
    df_batch.to_csv(batch_path, index=False)
    corrupted_batches[batch_name] = df_batch
    print(f"✅ {batch_name} generated")

print(f"\n✅ All 9 batches generated in {CORRUPT_DIR}")

In [ ]:
import src.corruption.inject as inject
from src.corruption.clean_num import clean_adult_dataset
X_copy = X.copy()

import pandas as pd

def all_numerical_corruptions_with_y(X, y, numeric_columns=None):
    X = X.copy()
    y = y.copy()
    if y.shape[1] != 1:
        raise ValueError("y must be a Series or single-column DataFrame")
    y_name = y.columns[0]
    y_df = y

    df = pd.concat([X, y_df], axis=1)

    if numeric_columns is None:
        numeric_columns = [
            c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])
        ]

    df_corrupted = inject.all_numerical_corruptions(
        df, columns=numeric_columns
    )

    X_corrupted = df_corrupted.drop(columns=[y_name])
    y_corrupted = df_corrupted[y_name]

    return X_corrupted, y_corrupted


X_corrupted, y_corrupted = all_numerical_corruptions_with_y(X_copy, y, numeric_features)
y_test, y_pred = fit_adult(X_corrupted, y_corrupted)


print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


# batches_config = inject.test_functions

# corrupted_batches = {}

# for batch_name, corruption_fn, kwargs in batches_config:
#     print(f"\n🔧 Batch {batch_name.split('_')[0]}: {batch_name.replace('_', ' ').title()}")
#     df_batch = corruption_fn(df_corrupt_source, **kwargs)
#     batch_path = os.path.join(CORRUPT_DIR, f"batch_{batch_name}.csv")
#     df_batch.to_csv(batch_path, index=False)
#     corrupted_batches[batch_name] = df_batch

# print(f"\n✅ All 9 batches generated in {CORRUPT_DIR}")

# corruption_results = {}

# for batch_name, df_batch in corrupted_batches.items():

#     X_corrupt = df_batch[selected_features]
#     y_corrupt = df_batch["income"]

#     # Split corrupted data (80% train, 20% test)
#     X_corrupt_train, X_corrupt_test, y_corrupt_train, y_corrupt_test = train_test_split(
#         X_corrupt, y_corrupt,
#         test_size=0.2,
#         random_state=42,
#         stratify=y_corrupt
#     )

# y_test, y_pred = fit_adult(df_corrupted, y)
# print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# print("Cleaning stats:", clean_stats)


### Data cleaning

In [ ]:
import src.corruption.clean_num as clean_num 
clf = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', HistGradientBoostingClassifier(
            learning_rate=0.06,
            max_depth=5,
            random_state=42
        ))
    ])
df_combined = pd.concat([X_corrupted, y_corrupted], axis=1)

cleaned=clean_num.clean_adult_dataset(df_combined, clf, numeric_cols=numeric_features)

In [ ]:
cleaned_df, model_stats = cleaned  # unpack

label_col = "income"  # same as used in cleaning

# Split into features and target
X_cleaned = cleaned_df.drop(columns=[label_col])
y_cleaned = cleaned_df[label_col].copy()

# Fit and evaluate
y_test, y_pred = fit_adult(X_cleaned, y_cleaned)


print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### Retraining and evaluation

In [ ]:
from src.corruption.clean import clean_all

"""
Refit baseline model on each corrupted batch and evaluate accuracy
"""
print("\n" + "="*70)
print("CELL 4: EVALUATE MODEL ON CORRUPTED BATCHES")
print("="*70)

# Store results
corruption_results = {}

for batch_name, df_batch in corrupted_batches.items():

    df_batch_clean = clean_all(df_batch)

    X_corrupt = df_batch_clean["text"].values
    y_corrupt = df_batch_clean["label"].values

    # Split corrupted data (80% train, 20% test)
    X_corrupt_train, X_corrupt_test, y_corrupt_train, y_corrupt_test = train_test_split(
        X_corrupt, y_corrupt,
        test_size=0.2,
        random_state=42,
        stratify=y_corrupt
    )
    
    print(f"   Train samples: {len(X_corrupt_train)}")
    print(f"   Test samples: {len(X_corrupt_test)}")
    
    # Train new model on corrupted data
    corrupted_model = build_model()
    corrupted_model.fit(X_corrupt_train, y_corrupt_train)
    
    # Evaluate on corrupted test set
    y_pred_corrupt = corrupted_model.predict(X_corrupt_test)
    corrupt_acc = accuracy_score(y_corrupt_test, y_pred_corrupt)
    
    print(f"\n   📊 Model Trained on Corrupted Data")
    print(f"   Accuracy: {corrupt_acc:.4f}")
    
    # Store results
    corruption_results[batch_name] = {
        "accuracy": corrupt_acc,
        "train_size": len(X_corrupt_train),
        "test_size": len(X_corrupt_test),
        "predictions": y_pred_corrupt,
        "true_labels": y_corrupt_test,
        "model": corrupted_model
    }
    
    # Compare to baseline
    accuracy_drop = baseline_acc - corrupt_acc
    drop_percentage = (accuracy_drop / baseline_acc) * 100
    print(f"\n   📉 Comparison to Baseline:")
    print(f"      Baseline Accuracy: {baseline_acc:.4f}")
    print(f"      Drop: {accuracy_drop:.4f} ({drop_percentage:.2f}%)")
    
    if drop_percentage > 10:
        print(f"      ⚠️  SIGNIFICANT DROP - Corruption heavily impacts model")
    elif drop_percentage > 5:
        print(f"      ⚡ MODERATE DROP - Corruption has noticeable impact")
    else:
        print(f"      ✅ MINIMAL DROP - Model is robust to this corruption")

print(f"\n{'='*70}")
print(f"✅ All corrupted batches evaluated")
print(f"{'='*70}")